In [1]:

pip install langid


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re


from pyfranc import franc

from langdetect import detect
import langid
from os import listdir
from os.path import isfile, join

#pip install pyfranc
#pip install langdetect

# remove emoji
def emoji_remove(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return(emoji_pattern.sub(r'', text)) # no emoji
# Prefix Removal
def prefix_removal(content):
 
    return re.sub(r'@\S+', '', content)
# Emoji Removal

# URL Removal
def url_removal(content):
    return re.sub(r'http\S+', '', content)

def RT_removal(content):
    return content.replace('RT','',1)
#remove empty line from dataset
def clean_text_full(tweet):
    remove_prefex = prefix_removal(tweet)
    remove_RT = RT_removal(remove_prefex)
    remove_url = url_removal(remove_RT)
    # remove_emoji = clean(remove_url, no_emoji=True)
    remove_emoji = emoji_remove(remove_url)
    return remove_emoji

def clean_text_partial (tweet):
    remove_prefex = prefix_removal(tweet)
    remove_RT = RT_removal(remove_prefex)
    remove_url = url_removal(remove_RT)
    return remove_url


def get_clean_table(df,newfilename):
    for index,row in df.iterrows():
        tweet = row['text']
        if str(tweet) == 'nan':
            df.drop(index, inplace=True)
            continue
    
        clean_text_withEmoji = clean_text_partial(tweet)
        isNotEmpty = len(clean_text_withEmoji.strip()) > 0
        if isNotEmpty:
            pure_text = emoji_remove(clean_text_withEmoji)
            isEnglish = langid.classify(pure_text)[0] == 'en'
            if isEnglish and isNotEmpty:
                df.at[index,'text'] = clean_text_withEmoji
                continue
            df.drop(index, inplace=True)
            
            continue
        df.drop(index, inplace=True)
    
    df.to_csv("result/" + newfilename)
# use this for auto read files in a folder
def readAllCSVFilenames(folderName):
    fileNames = [f for f in listdir(folderName) if isfile(join(folderName, f))]
    return fileNames


In [3]:

fileNames = readAllCSVFilenames("MID_PART") # folder name need to be changed 
print(fileNames)
 


['.DS_Store', 'LATE_2021_12_country.csv']


In [4]:
# folder result stores all new generated / processed file
cleanedFiles = readAllCSVFilenames("result")
for filename in fileNames:
    if filename == ".DS_Store":
        continue
    # self-define the new generated file name
    newFilename = filename.split(".")[0] + "_en.csv"
    print(newFilename)
    # file path need to  be changed
    df = pd.read_csv("MID_PART/" + filename, lineterminator='\n')
    if newFilename not in cleanedFiles:
        get_clean_table(df,newFilename)



LATE_2021_12_country_en.csv
